In [2]:
// Created by a.luchynska@gmail.com ## Реализация алгоритма ближнего соседа
// + модификация (определние начального города из условия минимальной длины маршрута)
// Node.js version 7.8.0
const _ = require('lodash');
const fs = require('fs');
const math = require('mathjs');
const dataT = fs.readFileSync('xqf131.tsp');

let rawArray = dataT.toString().split('\r\n');
const sectionName = 'NODE_COORD_SECTION'; // составим список городов
let cities = rawArray.slice(rawArray.indexOf(sectionName) + 1, rawArray.length - 2);
cities = cities.map((row) => {
  let rowArray = row.split(' ');
  return { n: rowArray[0], x: rowArray[1], y: rowArray[2] }
});
const n = cities.length; // кол-во городов
let distance = math.ones(n, n); // создадим единичную матрицу

distance = distance.map((val, j) => { // вычислим расстояния между точками, и поместим их в матрицу
  if (j[0] !== j[1]) {
    return math.sqrt(math.square(cities[j[1]].x - cities[j[0]].x) + math.square(cities[j[1]].y - cities[j[0]].y));
  } else return Infinity; //
});
let min = 0;
let expreiments = []; // массив объектов, содержащий искомые маршруты и их длину
math.range(0, n).forEach((stIndex) => { // цикл, формирующий массив объектов @expreiments
  const beginIndex = stIndex;
  let stepIndex = beginIndex;
  let finalRoad = [{index: stepIndex, dist: 0}]; // объект, хранящий путь
  let tempDistance = _.cloneDeep(distance); // основной цикл работает с копией матрицы
  math.range(0, n).forEach((index) => { // основной цикл алгоритма
    let tempMatrix = tempDistance.subset(math.index(stepIndex, math.range(0, n)));
    let tempArr = tempMatrix._data[0];
    if (index !== (n - 1)) tempArr[beginIndex] = Infinity; // условие (1), гарантирующее обход всех вершин до замыкания
    min = math.min(tempArr); // находим минимум               Гамильтонового цикла
    tempArr.every((value, i) => {
      if (value === min) {
        if ((i === beginIndex) && (index !== (n - 1))) return true; // также условие (1)
          math.range(0, n).forEach((j) => { // помечаем отрезок пути как пройденный, т.е. запрещаем переход:
          tempDistance.subset(math.index(stepIndex, j), Infinity); // 1) от текущего города к остальным
          if (stepIndex !== beginIndex) tempDistance.subset(math.index(j, stepIndex), Infinity); // 2) от остальных городов к текущему(если город не начальный)
          tempDistance.subset(math.index(j, i), Infinity); // 3) от остальных к выбранному для посещению городу
        });
        tempDistance.subset(math.index(i, stepIndex), Infinity); // 4) от выбранного к текущему
        finalRoad.push({index: i, dist: value}); // заполняем путь новым отрезком
        stepIndex = i; // присваиваем новый текущий город
        return false;
      } else return true;
    })
  });
  let optimalValue = math.sum(finalRoad.map((el => el.dist)));
  console.log(`${beginIndex} - ${optimalValue}`);
  expreiments.push({ optimalValue, finalRoad } )
});
expreiments.sort((a, b) => {
  if (a.optimalValue > b.optimalValue) {
    return 1;
  }
  if (a.optimalValue < b.optimalValue) {
    return -1;
  }
  return 0;
});
let result = expreiments[0].finalRoad.map((value) => {
  return cities.find((el, i) => i === value.index);
});
console.log(expreiments[0].optimalValue);
let someString = '';
result.forEach((val, i) => someString += `${val.x} ${val.y}\r\n`);
fs.writeFileSync('result.txt', someString, (err) => {
  if (err) throw err;
  console.log('The file has been saved!');
});

0 - 709.5216278258143
1 - 696.9992495062722
2 - 695.7561136803014
3 - 695.4588545647349
4 - 690.4270557954928
5 - 706.6369958613982
6 - 701.4382903274097
7 - 699.7691069739003
8 - 697.5326419223038
9 - 759.5021887217931
10 - 698.4897114593587
11 - 692.1084045339137
12 - 700.2770491457647
13 - 685.5466187551465
14 - 685.5614816211363
15 - 685.5766471211057
16 - 696.936434320474
17 - 689.4041937069255
18 - 699.2426291608903
19 - 693.6114792163146
20 - 702.1734558617245
21 - 707.2854884001541
22 - 737.3435475580114
23 - 751.0393012834653
24 - 690.5469881776321
25 - 696.0301483956016
26 - 701.6844835290561
27 - 694.1270138247972
28 - 694.9349854097294
29 - 697.0347863220843
30 - 739.4491859159938
31 - 738.7314590203046
32 - 739.2696370754232
33 - 743.6187161947055
34 - 743.0356339930657
35 - 743.708558488875
36 - 746.8040264442035
37 - 746.3552844624052
38 - 747.1620442600383
39 - 711.4451459355322
40 - 710.8632039478158
41 - 713.2350782273047
42 - 712.4031863562088
43 - 712.3442024583685


undefined